In [1]:
!pip -q install colorlog
!pip -q install keras_radam
!pip -q install keras_tqdm
#!pip install tensorflow==2.1.0-rc1
!pip install tensorflow==2.1.0 

!pip install tensorflow_probability==0.8.0
# inference_adversarial/discr2gen_rate0.1_gen2trad_rate0.5/convolutional/vae

# Settings

In [2]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
/


In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [4]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

/


In [5]:
%cd /content/
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/

/content
Cloning into 'Generative_Models'...
remote: Enumerating objects: 2037, done.
remote: Counting objects: 100% (2037/2037), done.
remote: Compressing objects: 100% (561/561), done.
remote: Total 14861 (delta 1480), reused 1885 (delta 1359), pack-reused 12824
Receiving objects: 100% (14861/14861), 414.65 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (10081/10081), done.
Checking out files: 100% (1674/1674), done.
/content/Generative_Models


In [6]:
os.chdir('/content/Generative_Models/')
print(os.getcwd())

/content/Generative_Models


# Dataset loading

In [7]:
dataset_name = 'dsprites'
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='dsprites',
    version=2.0.0,
    description='dSprites is a dataset of 2D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *color*, *shape*, *scale*,
*rotation*, *x* and *y* positions of a sprite.

All possible combinations of these latents are present exactly once,
generating N = 737280 total images.

### Latent factor values

*   Color: white
*   Shape: square, ellipse, heart
*   Scale: 6 values linearly spaced in [0.5, 1]
*   Orientation: 40 values in [0, 2 pi]
*   Position X: 32 values in [0, 1]
*   Position Y: 32 values in [0, 1]

We varied one latent at a time (starting from Position Y, then Position X, etc),
and sequentially stored the images in fixed order.
Hence the order along the first dimension is fixed and allows you to map back to
the value of the latents corresponding to that image.

We chose the latents values deliberately to have the smallest step changes
while ensuring that all pixel outp

In [0]:
intermediate_dim = 20
inputs_shape=(64, 64, 1) # image shape
batch_size = 32
latents_dim = 6

In [0]:
TRAIN_BUF = 600
TEST_BUF = 100

In [10]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/dsprites/2.0.0.incompleteWXBQK6/dsprites-train.tfrecord


Dataset dsprites downloaded and prepared to /root/tensorflow_datasets/dsprites/2.0.0. Subsequent calls will reuse this data.


In [0]:
from data.gt_load.datasets import load

In [12]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

--2020-05-28 18:08:44--  https://github.com/deepmind/dsprites-dataset/raw/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz [following]
--2020-05-28 18:08:45--  https://raw.githubusercontent.com/deepmind/dsprites-dataset/master/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19318135 (18M) [application/octet-stream]
Saving to: ‘/content/Generative_Models/data/.gt_datasets/dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.n

In [13]:
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.gt_datasets'
#absolute = abspath(DATA_PATH)
print(DATA_PATH)

/content/Generative_Models/data/.gt_datasets


In [14]:
!ls /content/Generative_Models/data/.gt_datasets/dsprites

dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz


In [0]:
eval_dataset = load(dataset_name='dsprites_full',dataset_path=DATA_PATH)

In [0]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'].numpy().max())
    break

In [17]:
_instance_scale

1.0

In [0]:
import numpy as np
from collections.abc import Iterable

In [0]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [20]:
_outputs_shape

4096

# Model's Layers definition

In [0]:
units=intermediate_dim
c=50
menc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=units, kernel_size=1, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latents_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=units*c*c, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, units)),
    tf.keras.layers.Conv2DTranspose(filters=units*3, kernel_size=3, strides=(4, 4), padding="SAME", activation='relu'),
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [22]:
from google.colab import drive
drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/DstResults/
MAIN_SAVE_DIR = '/content/drive/My Drive/DstResults'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model_name = dataset_name+'GAVAE_Dense_reconst_ell'
#experiments_dir='experiments'+sep_local+model_name
recording_dir=MAIN_SAVE_DIR + sep_local  +'recording-Dense'+sep_local + model_name

In [0]:
from training.adversarial_basic.generative_adversarial.autoencoders.VAAE import VAAE as AE

In [0]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist

In [0]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [28]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/DstResults/recording-Dense/dspritesGAVAE_Dense_reconst_ell/var_save_dir'

In [0]:
#to restore trained model, set filepath=_restore

In [29]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

  WARNING  | None
  WARNING  | None
  WARNING  | None


Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 20)        40        
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 122886    
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
activity_regularization (Act (None, 6)              

TypeError: ignored

In [0]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses
    

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'generative_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [0]:
#ae.compile(metrics=None)
discr2gen_rate = 0.001
gen2trad_rate = 0.1

ae.compile(
    adversarial_losses=generative_discriminator_losses,
    adversarial_weights={'generator_weight': gen2trad_rate, 'discriminator_weight': discr2gen_rate}
)

# Callbacks

In [0]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [0]:
gts_csv = os.path.join(recording_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(recording_dir, 'csv_dir', 'gtu_metrics')

In [0]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=10
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    gt_freq=10
)

In [0]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [34]:
ms = ModelSaver(filepath=_restore,save_freq=5)

TypeError: ignored

In [35]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

NameError: ignored

In [0]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [36]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

NameError: ignored

In [0]:
import numpy as np

# Model Training

In [38]:
ae.fit(
    x=train_ds,
    input_kw='image',
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)

NameError: ignored

# Model Evaluation

## inception_score

In [0]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [0]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [0]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [0]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [0]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [0]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [0]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [0]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [0]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [0]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_randomly

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [0]:
from training.generators.image_generation_testing import interpolate_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)